In [ ]:
'''

Para hacernos una idea de como actua el soporte en el entorno de NuScenes vamos
a realizar el histograma de los radios que obtenemos a lo largo de todo el CAN-bus.
Posteriormente definiremos 3 tipos de curva:

->Amplia
->Intermedia
->Cerrada

Posteriormente haremos un render para ver como se comporta, viendo así los puntos
de LiDAR que entran en una cámara de FOV=60º con soporte y sin soporte. Además,
Probaremos para distintas longitudes de arco de curva (L).

'''

In [32]:
import os
import json
import glob
import pandas as pd

# Directorio donde se encuentran los archivos JSON
directory = r"C:\Users\usuario\Desktop\CAF\v1.0-mini_canbus-001\v1.0-mini\can_bus\can_bus"

data = []

# Buscar todos los archivos que sigan el patrón "scene-*_pose.json"
pattern = os.path.join(directory, "scene-*_pose.json")
for filepath in glob.glob(pattern):
    with open(filepath, "r") as f:
        # Se asume que el archivo contiene una lista de registros JSON
        records = json.load(f)
    # Extraer el identificador de escena a partir del nombre del archivo.
    filename = os.path.basename(filepath)
    scene = filename.split("_pose.json")[0]
    
    # Recorrer cada registro en el archivo
    for rec in records:
        try:
            # Extraer: aceleración en y (posición 1 de "accel"), 
            # rotation rate en z (posición 2 de "rotation_rate")
            # velocidad en x (posición 0 de "vel")
            acceleration_y = rec["accel"][1]
            rotation_rate_z = rec["rotation_rate"][2]
            velocity_x = rec["vel"][0]
        except Exception as e:
            print(f"Error procesando registro en {filename}: {e}")
            continue
        
        data.append({
            "scene": scene,
            "acceleration_y": acceleration_y,
            "rotation_rate_z": rotation_rate_z,
            "vel_x": velocity_x
        })

# Crear un DataFrame con los datos extraídos y guardarlo en una variable
df_canbus = pd.DataFrame(data)

# Imprimir las primeras filas del DataFrame
print(df_canbus.head())

        scene  acceleration_y  rotation_rate_z     vel_x
0  scene-0001        0.929175         0.284921  4.168876
1  scene-0001        0.976245         0.287632  4.168600
2  scene-0001        1.026607         0.289735  4.168416
3  scene-0001        1.079439         0.291279  4.167358
4  scene-0001        1.133072         0.292854  4.166189


In [37]:
import pandas as pd
from scipy.signal import butter, filtfilt

# --- Definir filtro Butterworth 2 Hz ---
fs = 100       # Frecuencia de muestreo (Hz)
order = 4
cutoff = 2.0
b, a = butter(order, cutoff / (0.5 * fs), btype='low')

# --- Aplicar el filtro a la aceleración (en y) y a la velocidad angular (rotation_rate en z) ---
# Se asume que 'df_canbus' ya tiene las columnas "acceleration_y", "rotation_rate_z" y "vel_x"
df_canbus['accel_cent_filt'] = filtfilt(b, a, df_canbus['acceleration_y'])
df_canbus['rotation_rate_z_filt'] = filtfilt(b, a, df_canbus['rotation_rate_z'])

# --- Funciones para calcular radios (sin cambios) ---
def calcular_radio_por_rot_rate(df):
    df['radio_R_rot'] = df.apply(
        lambda row: row['vel_x'] / row['rotation_rate_z_filt']
        if pd.notna(row['rotation_rate_z_filt']) and abs(row['rotation_rate_z_filt']) > 1e-2 and abs(row['accel_cent_filt']) >= 0.75
        else None,
        axis=1
    )
    return df

def calcular_radio_por_aceleracion(df):
    df['radio_R'] = df.apply(
        lambda row: row['vel_x']**2 / row['accel_cent_filt']
        if abs(row['accel_cent_filt']) >= 0.75
        else None,
        axis=1
    )
    return df

# Aplicar las funciones calculadoras sobre el DataFrame
df_canbus = calcular_radio_por_rot_rate(df_canbus)
df_canbus = calcular_radio_por_aceleracion(df_canbus)

# Una vez calculados los radios, quedarnos con su valor absoluto
df_canbus['radio_R_rot'] = df_canbus['radio_R_rot'].abs()
df_canbus['radio_R'] = df_canbus['radio_R'].abs()

# Imprimir las primeras filas del DataFrame resultante
print(df_canbus.head())

        scene  acceleration_y  rotation_rate_z     vel_x  accel_cent_filt  \
0  scene-0001        0.929175         0.284921  4.168876         0.924919   
1  scene-0001        0.976245         0.287632  4.168600         0.954378   
2  scene-0001        1.026607         0.289735  4.168416         0.983677   
3  scene-0001        1.079439         0.291279  4.167358         1.012654   
4  scene-0001        1.133072         0.292854  4.166189         1.041148   

   rotation_rate_z_filt  radio_R_rot    radio_R  
0              0.283171    14.722124  18.790327  
1              0.285929    14.579158  18.207905  
2              0.288730    14.437064  17.664017  
3              0.291569    14.292882  17.149868  
4              0.294438    14.149639  16.671140  


In [ ]:
'''SIGUE AQUI EL SIGUIENTE DIA'''

In [3]:
import os
import glob
import cv2
import matplotlib.pyplot as plt
from vcd import core, utils, draw, scl

MAX_DISTANCE = 50.0
BEV_HEIGHT = 1024
BEV_WIDTH = 1024
CS_REF = "train"

class BEVGenerator:
    def __init__(self, cam_name, img_path, openlabel_path, output_path):
        self.camera_name    = cam_name
        self.img_path       = img_path
        self.openlabel_path = openlabel_path
        self.output_path    = output_path

        # Cargo la imagen
        self.img = cv2.imread(self.img_path)
        if self.img is None:
            raise FileNotFoundError(f"No se pudo leer la imagen: {self.img_path}")

        # Cargo anotaciones y construyo la escena
        self.vcd = core.VCD()
        self.vcd.load_from_file(self.openlabel_path)
        self.scene = scl.Scene(self.vcd)

        # Parámetros BEV
        aspect = BEV_WIDTH / BEV_HEIGHT
        x_range = (0.0, MAX_DISTANCE)
        y_half = (x_range[1] - x_range[0]) / aspect / 2
        y_range = (-y_half, y_half)

        self.params = draw.TopView.Params(
            color_map=utils.COLORMAP_1,
            topview_size=(BEV_WIDTH, BEV_HEIGHT),
            background_color=0,
            range_x=x_range,
            range_y=y_range,
            step_x=1.0,
            step_y=1.0,
            draw_grid=True
        )

    def src_img2bev(self, frame_num: int = 0):
        drawer = draw.TopView(
            scene=self.scene,
            coordinate_system=CS_REF,
            params=self.params
        )
        drawer.add_images(imgs={self.camera_name: self.img}, frame_num=frame_num)
        drawer.draw_bevs(_frame_num=frame_num)

        bev_img = drawer.topView
        os.makedirs(os.path.dirname(self.output_path), exist_ok=True)
        cv2.imwrite(self.output_path, bev_img)
        print(f"[OK] {os.path.basename(self.img_path)} → {os.path.basename(self.output_path)}")


def main():
    base_dir   = r"C:/Users/usuario/Desktop/CAF/VS_CAF/CAF/Data_3D_2D/Train"
    images_dir = os.path.join(base_dir, "input", "images")
    label_dir  = os.path.join(base_dir, "input", "label")
    output_dir = os.path.join(base_dir, "output")

    # Buscamos un JSON (por ejemplo "openlabel.json") dentro de label_dir
    json_files = glob.glob(os.path.join(label_dir, "*.json"))
    if not json_files:
        print("No hay ningún .json en:", label_dir)
        return
    # Si hay varios JSON, puedes ajustar aquí cómo elegir cuál usar
    openlabel_path = json_files[0]
    print("Usando etiquetas de:", os.path.basename(openlabel_path))

    # Recogemos todas las imágenes (.png/.jpg)
    img_patterns = ["*.png", "*.jpg", "*.jpeg"]
    image_paths = []
    for p in img_patterns:
        image_paths.extend(glob.glob(os.path.join(images_dir, p)))
    if not image_paths:
        print("No se encontraron imágenes en:", images_dir)
        return

    # Procesamos cada imagen con el mismo JSON
    for img_path in sorted(image_paths):
        name, _ = os.path.splitext(os.path.basename(img_path))
        output_path = os.path.join(output_dir, f"{name}_topview.png")

        bev = BEVGenerator(
            cam_name="cam",
            img_path=img_path,
            openlabel_path=openlabel_path,
            output_path=output_path
        )
        bev.src_img2bev(frame_num=0)

    print("✔️  Procesadas todas las imágenes con el mismo JSON.")

if __name__ == "__main__":
    main()


Usando etiquetas de: tren1.json
cam top view remap computation...
[OK] tren1.png → tren1_topview.png
✔️  Procesadas todas las imágenes con el mismo JSON.


In [12]:
import os
import cv2
import matplotlib.pyplot as plt
from nuscenes.nuscenes import NuScenes
from vcd import core, utils, draw, scl

# === Parámetros BEV ===
MAX_DISTANCE = 50.0
BEV_HEIGHT   = 1024
BEV_WIDTH    = 1024
CS_REF       = "vehicle-iso8855"
scene_name  = "scene-0916"

class BEVGenerator():
    def __init__(self, cam_name, img_path, openlabel, output):
        self.camera_name = cam_name        
        self.img = cv2.imread(img_path)
        self.vcd = core.VCD()
        self.vcd.load_from_file(openlabel)
        self.scene = scl.Scene(self.vcd)
        self.output_folder = output

        bev_aspect_ratio = BEV_WIDTH / BEV_HEIGHT
        bev_x_range = (0.0, MAX_DISTANCE)
        bev_y_range = (
            -((bev_x_range[1] - bev_x_range[0]) / bev_aspect_ratio) / 2,
             ((bev_x_range[1] - bev_x_range[0]) / bev_aspect_ratio) / 2
        )
        self.bev_parameters = draw.TopView.Params(
            color_map       = utils.COLORMAP_1,
            topview_size    = (BEV_WIDTH, BEV_HEIGHT),
            background_color= 0,
            range_x         = bev_x_range,
            range_y         = bev_y_range,
            step_x          = 1.0,
            step_y          = 1.0,
            draw_grid       = True
        )

    def src_img2bev(self):
        drawer = draw.TopView(
            scene             = self.scene,
            coordinate_system = CS_REF,
            params            = self.bev_parameters
        )
        drawer.add_images(imgs={self.camera_name: self.img}, frame_num=0)
        drawer.draw_bevs(_frame_num=0)
        bev_img = drawer.topView
        cv2.imwrite(self.output_folder, bev_img)


def main(camera_name, image_path, openlabel_path, output_path):
    bev = BEVGenerator(
        cam_name   = camera_name,
        img_path   = image_path,
        openlabel  = openlabel_path,
        output     = output_path
    )
    bev.src_img2bev()


# === Rutas ===
DATASET_PATH  = "C:/Users/usuario/Desktop/CAF/v1.0-mini_canbus-001"
OPENLABEL     = "C:/Users/usuario/Desktop/CAF/VS_CAF/CAF/Data_3D_2D/NuScenes/input/label/coche1.json"
OUTPUT_DIR    = "C:/Users/usuario/Desktop/CAF/VS_CAF/CAF/Data_3D_2D/NuScenes/output_bev"
os.makedirs(OUTPUT_DIR, exist_ok=True)

# === Inicializa NuScenes y busca escena 0916 ===
nusc = NuScenes(version='v1.0-mini', dataroot=DATASET_PATH, verbose=False)
scene = next(s for s in nusc.scene if s['name']==scene_name)
token = scene['first_sample_token']

# === Itera sobre los primeros 100 samples de CAM_FRONT de esa escena ===
contador = 0
while token and contador < 100:
    sample = nusc.get('sample', token)
    if 'CAM_FRONT' in sample['data']:
        sd     = nusc.get('sample_data', sample['data']['CAM_FRONT'])
        img_rel= sd['filename']
        img_abs= os.path.join(nusc.dataroot, img_rel)
        salida = os.path.join(OUTPUT_DIR, f"bev_{contador:03d}.png")

        print(f"[{contador+1:03d}] Procesando {os.path.basename(img_abs)} → {salida}")
        main(
            camera_name    = "camera",
            image_path     = img_abs,
            openlabel_path = OPENLABEL,
            output_path    = salida
        )
        contador += 1

    token = sample['next']

print(f"Terminado: se han generado {contador} imágenes BEV en\n{OUTPUT_DIR}")


[001] Procesando n015-2018-10-08-15-36-50+0800__CAM_FRONT__1538984233512470.jpg → C:/Users/usuario/Desktop/CAF/VS_CAF/CAF/Data_3D_2D/NuScenes/output_bev\bev_000.png
camera top view remap computation...
[002] Procesando n015-2018-10-08-15-36-50+0800__CAM_FRONT__1538984234012472.jpg → C:/Users/usuario/Desktop/CAF/VS_CAF/CAF/Data_3D_2D/NuScenes/output_bev\bev_001.png
camera top view remap computation...
[003] Procesando n015-2018-10-08-15-36-50+0800__CAM_FRONT__1538984234512460.jpg → C:/Users/usuario/Desktop/CAF/VS_CAF/CAF/Data_3D_2D/NuScenes/output_bev\bev_002.png
camera top view remap computation...
[004] Procesando n015-2018-10-08-15-36-50+0800__CAM_FRONT__1538984234912460.jpg → C:/Users/usuario/Desktop/CAF/VS_CAF/CAF/Data_3D_2D/NuScenes/output_bev\bev_003.png
camera top view remap computation...
[005] Procesando n015-2018-10-08-15-36-50+0800__CAM_FRONT__1538984235412460.jpg → C:/Users/usuario/Desktop/CAF/VS_CAF/CAF/Data_3D_2D/NuScenes/output_bev\bev_004.png
camera top view remap compu